In [1]:
import numpy as np
import tensorflow as tf
import os

from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, ITURadioMaterial, PathSolver

import mitsuba as mi
import drjit as dr

%matplotlib
import matplotlib.pyplot as plt

# mi.set_variant('scalar_rgb')

2025-04-22 21:07:44.605422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745370464.622064  260840 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745370464.627034  260840 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745370464.640458  260840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745370464.640477  260840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745370464.640479  260840 computation_placer.cc:177] computation placer alr

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
min = (0,0, 0)
max = (275, 275, 300)

h_spacing = 25
v_spacing = 25
num_tx = 1

scene_dir = "minis3"
scene_files = [f for f in os.listdir(scene_dir) if f.endswith('.xml')]
print(scene_files)

['row15_col13.xml', 'row0_col4.xml', 'row5_col1.xml', 'row2_col14.xml', 'row7_col0.xml', 'row2_col5.xml', 'row5_col8.xml', 'row3_col14.xml', 'row5_col6.xml', 'row0_col1.xml', 'row2_col11.xml', 'row3_col5.xml', 'row8_col7.xml', 'row8_col2.xml', 'row9_col3.xml', 'row1_col2.xml', 'row4_col2.xml', 'row15_col10.xml', 'row5_col7.xml', 'row10_col8.xml', 'row10_col7.xml', 'row8_col15.xml', 'row8_col0.xml', 'row4_col9.xml', 'row10_col12.xml', 'row6_col13.xml', 'row7_col4.xml', 'row10_col13.xml', 'row3_col10.xml', 'row0_col9.xml', 'row14_col8.xml', 'row11_col13.xml', 'row5_col15.xml', 'row2_col3.xml', 'row15_col9.xml', 'row4_col6.xml', 'row5_col0.xml', 'row0_col0.xml', 'row10_col10.xml', 'row11_col8.xml', 'row1_col8.xml', 'row2_col1.xml', 'row14_col15.xml', 'row2_col4.xml', 'row0_col6.xml', 'row11_col15.xml', 'row7_col8.xml', 'row5_col9.xml', 'row10_col3.xml', 'row0_col5.xml', 'row8_col13.xml', 'row4_col10.xml', 'row9_col6.xml', 'row0_col11.xml', 'row6_col11.xml', 'row15_col8.xml', 'row4_col0.xm

In [3]:
scene_file = scene_files[10]
scene = load_scene(os.path.join(scene_dir, scene_file))
output_dir = os.path.splitext(scene_file)[0]

try:
    os.mkdir(output_dir)
except OSError:
    print('dir already exists')

In [4]:
rx_points = []

x_start, y_start, _ = min
x_end, y_end, _ = max
xx = np.linspace(x_start, x_end, h_spacing)
yy = np.linspace(y_start, y_end, h_spacing)
ray_pos = np.array([(x, y, 1000) for x in xx for y in yy])

z_rays = mi.Ray3f(o=ray_pos.T, d=(0, 0, -1))
z_hits = scene.mi_scene.ray_intersect(z_rays)

for z_hit, valid in zip(z_hits.p.numpy().T, z_hits.is_valid()):
    if (not valid):
        continue
    # start = int(np.ceil(z_hit[2] / spacing) * spacing)
    start = int(z_hit[2] + 10)
    
    # print(start)
    for i in range(start, 100, v_spacing):
        # print([z_hit[0], z_hit[1], i])
        rx_points.append(np.array([z_hit[0], z_hit[1], i]))
    # rx_points.append(z_hit)
# print(rx_points)

In [5]:
## Include points that did not have a valid hitpoint due to lack of floor right now
for i, v in enumerate(z_hits.is_valid()):
    if(not v):
        r = ray_pos[i]
        for i in range(5, 100, v_spacing):
            rx_points.append(np.array([r[0], r[1], i]))
        # rx_points.append(np.array([r[0], r[1], 0]))

In [6]:
scene.frequency # Defaults to 3.5GHz

mat = ITURadioMaterial(name='concrete', itu_type='concrete', thickness=0.1, scattering_coefficient=0.0, xpd_coefficient=0.0)

# Set all objects to concrete for now
for obj in scene.objects.values():
    obj.radio_material = mat

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=1,
                          horizontal_spacing=1,
                          polarization="V",
                          pattern="iso")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

# Create transmitter
# tx = Transmitter(name="tx",
#               position=[200,200,100],
#               orientation=[0,0,0])
# scene.add(tx)

In [7]:
# Create receivers
for rx_i in range(len(rx_points)):
#     print(rx_points[rx_i])
    rx = Receiver(name=f"rx_{rx_i}",
            position=rx_points[rx_i],
            orientation=[0,0,0])
    scene.add(rx)

In [8]:
# tx_pos = []
# for tx_i in range(num_tx):
    
points = np.array(rx_points)
filtered = points[(points[:,2] <= 50)]
# print(filtered)
filtered[:,2] += 5
# print(filtered)

np.random.seed(42)
c = np.random.choice(filtered.shape[0], size=num_tx, replace=False)
print(c)

tx_pos = filtered[c]

for tx_i in range(num_tx):
    tx = Transmitter(name=f"tx_{tx_i}",
            position=tx_pos[tx_i],
            orientation=[0,0,0])
    scene.add(tx)

[1177]


In [9]:
p_solver  = PathSolver()
paths = p_solver(scene=scene,
                 max_depth=5,
                 los=False,
                 specular_reflection=True,
                 diffuse_reflection=False,
                 refraction=False,
                 synthetic_array=False,
                 seed=41)

In [10]:
# import cProfile
# p_solver  = PathSolver()

# cProfile.run(p_solver(scene=scene,
#                  max_depth=5,
#                  los=False,
#                  specular_reflection=True,
#                  diffuse_reflection=False,
#                  refraction=False,
#                  synthetic_array=False,
#                  seed=41))


In [11]:
scene.preview(paths=paths)
None

Export fields to csv's

transmitter locs
receiver locs
a
primitives
pickle paths obj


In [12]:
def log_flat_arr(file, X, fmt='%u'):
    shape = X.shape
    flattened = X.reshape(shape[0], -1)
    np.savetxt(fname=os.path.join(output_dir, file), X=flattened, header=f'{shape}', fmt=fmt)


In [13]:
# scene.transmitters.
tx_log = paths.sources.numpy().T
log_flat_arr("tx_locations", tx_log)
# np.savetxt(fname=os.path.join(output_dir, "tx_locations"), X=tx_log)

rx_log = paths.targets.numpy().T
log_flat_arr("rx_locations", rx_log)

# np.savetxt(fname=os.path.join(output_dir, "rx_locations"), X=rx_log)

real_a_log = paths.a[0].numpy()
# print(real_a_log.shape)
imaginary_a_log = paths.a[1].numpy()
# print(real_a_log.reshape(real_a_log.shape[0], -1).shape)
# real_a_log = real_a_log.reshape(real_a_log.shape[0], -1)
# imaginary_a_log = imaginary_a_log.reshape(imaginary_a_log.shape[0], -1)
log_flat_arr("real_a", real_a_log)
log_flat_arr("imaginary_a", imaginary_a_log)



# np.savetxt(fname=os.path.join(output_dir, "real_a"), X=real_a_log)
# np.savetxt(fname=os.path.join(output_dir, "imaginary_a"), X=imaginary_a_log)

# paths.valid.numpy().shape

# paths.a
# paths.valid.shape
valid_log = paths.valid.numpy()

# valid_shape = valid_log.shape
# valid_log = valid_log.reshape(valid_log.shape[0], -1)

# np.savetxt(fname=os.path.join(output_dir, "valid"), X=valid_log, header=f'{valid_shape}')
log_flat_arr("valid", valid_log)

primitives_log = paths.primitives.numpy()
log_flat_arr("primitives", primitives_log)

vertices_log = paths.primitives.numpy()
log_flat_arr("vertices", vertices_log)



In [14]:
type(valid_log)

numpy.ndarray

In [23]:
read_in = np.genfromtxt(fname=os.path.join(output_dir, "valid"), dtype=int)
# np.array_equal(read_in.reshape(4842, 1, 2, 1, 8), paths.a[0].numpy())
read_in

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0]])